# `raman_rabi` documentation

Here are instructions and examples for using the `raman_rabi` package developed by Taylor Patti, Jamelle Watson-Daniels, and Soumya Ghosh for Prof. Manoharan's PHYS 201: Statistics for Physicists course at Harvard.

## Signs of life

Having installed the package, we first go to the root directory and run the unit tests:

In [1]:
!nosetests

.....
----------------------------------------------------------------------
Ran 5 tests in 0.264s

OK


Now try importing the package itself and running its test function:

In [2]:
import raman_rabi

print(raman_rabi.testing.hello())

>>> Hello from raman_rabi!


## Week 1 Model

We want to know if the functional form of the electronic-nuclear flip-flop transition in the Raman-Rabi NV center protocol [arXiv:1808.04346]. The data we are interested in is the change in $m_N = +1$ and $m_s = 0$ nuclear spin populations as a function of Raman-Rabi pulse length in Fig. 4.

It is hypothesized by the paper that this behavior is weakly driven, over-damped Raman-Rabi oscillations. That is, partial coherent transitions between the two nuclear spin populations marred by various sources of state decay and decoherence. The ideal theoretical model proposed for the nuclear spin specific fluorescence for spin $m_N$ is, $I_T^{m_N}(t)$, is described by

$$I_T^{m_N}(t) = B_G + A_p e^{-\Gamma_p t} + A_h \cos\left( \Omega_h t \right) e^{-\Gamma_{deph} t},$$

where $B_G$ is the background population/fluorescence, $A_p$ is the strength at which parasitic transitions remove states from the cycle (i.e. into $m_s = -1$ or $m_N = -1$ states, $\Gamma_p$ is the rate at which this parasitic loss occurs, $A_h$ is the strength at which the hyperfine interaction induces this transition, $\Omega_h$ is the rate at which the hyperfine transition occurs, and $\Gamma_{deph}$ is the rate of dephasing in this coherent oscillation.

This week, we will be focusing on a basic analysis where we will average over all of our data points for each $m_S$ and then consider our full signal model as

$$I(t) = I_T^{m_N}(t) + \eta_{shot}(t)$$

where $\eta_{shot}(t)$ is a random variable describing the measurement shot noise (Poissonian distribution) of mean $\lambda = \sqrt{I_T^{m_N}(t)}$ is. This is because the expectation value of shot noise is equal to the square root of the ideal signal, the shot noise is described by a Poissonian distribution, and the expectation value of the Poissonian distribution is its mean. HOWEVER, as the fluoresence data is taken as a continuum (not single photon counting) we will take the limit of large counts of a Poissonian distribution and obtain a Gaussian distribution for the shot noise random variable.